# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe274adf7f0>
├── 'a' --> tensor([[0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867]])
└── 'x' --> <FastTreeValue 0x7fe274adf5e0>
    └── 'c' --> tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                        [-0.3343,  1.6618, -0.6957,  0.5624],
                        [-1.9314, -0.3298,  0.5371, -0.9942]])

In [4]:
t.a

tensor([[0.0454, 0.3209, 1.5625],
        [1.1581, 1.4147, 2.6867]])

In [5]:
%timeit t.a

62.9 ns ± 0.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe274adf7f0>
├── 'a' --> tensor([[-0.3411,  0.0506,  0.1972],
│                   [ 0.7969, -0.8285,  0.0570]])
└── 'x' --> <FastTreeValue 0x7fe274adf5e0>
    └── 'c' --> tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                        [-0.3343,  1.6618, -0.6957,  0.5624],
                        [-1.9314, -0.3298,  0.5371, -0.9942]])

In [7]:
%timeit t.a = new_value

66.5 ns ± 0.0538 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.0454, 0.3209, 1.5625],
               [1.1581, 1.4147, 2.6867]]),
    x: Batch(
           c: tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942]]),
       ),
)

In [10]:
b.a

tensor([[0.0454, 0.3209, 1.5625],
        [1.1581, 1.4147, 2.6867]])

In [11]:
%timeit b.a

59.6 ns ± 0.0196 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9340,  0.0407,  0.4915],
               [-1.3983, -1.7783,  0.8816]]),
    x: Batch(
           c: tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.163 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 2.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 514 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 689 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe274b4bdc0>
├── 'a' --> tensor([[[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]],
│           
│                   [[0.0454, 0.3209, 1.5625],
│                    [1.1581, 1.4147, 2.6867]]])
└── 'x' --> <FastTreeValue 0x7fe274ad77f0>
    └── 'c' --> tensor([[[ 0.6068,  1.7537,  0.0644, -1.8648],
           

In [21]:
%timeit t_stack(trees)

32.6 µs ± 74.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe274ad72e0>
├── 'a' --> tensor([[0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867],
│                   [0.0454, 0.3209, 1.5625],
│                   [1.1581, 1.4147, 2.6867]])
└── 'x' --> <FastTreeValue 0x7fe1bfdbbdf0>
    └── 'c' --> tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                        [-0.3343,  1.6618, -0.6957,  0.5624],
                        [-1.9314, -0.3298,  0.5371, -0.9942],
   

In [23]:
%timeit t_cat(trees)

29.9 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6068,  1.7537,  0.0644, -1.8648],
                       [-0.3343,  1.6618, -0.6957,  0.5624],
                       [-1.9314, -0.3298,  0.5371, -0.9942]],
              
                      [[ 0.6068,  1.7537,  0.0644, -1.8648],
                       [-0.3343,  1.6618, -0.6957,  0.5624],
                       [-1.9314, -0.3298,  0.5371, -0.9942]],
              
                      [[ 0.6068,  1.7537,  0.0644, -1.8648],
                       [-0.3343,  1.6618, -0.6957,  0.5624],
                       [-1.9314, -0.3298,  0.5371, -0.9942]],
              
                      [[ 0.6068,  1.7537,  0.0644, -1.8648],
                       [-0.3343,  1.6618, -0.6957,  0.5624],
                       [-1.9314, -0.3298,  0.5371, -0.9942]],
              
                      [[ 0.6068,  1.7537,  0.0644, -1.8648],
                       [-0.3343,  1.6618, -0.6957,  0.5624],
                       [-1.9314, -0.3298,  0.5371, -0.9942]],

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942],
                      [ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942],
                      [ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942],
                      [ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942],
                      [ 0.6068,  1.7537,  0.0644, -1.8648],
                      [-0.3343,  1.6618, -0.6957,  0.5624],
                      [-1.9314, -0.3298,  0.5371, -0.9942],
                      [ 0.6068,  1.7537,  0.0644, -1.8648],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

334 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
